In [104]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, GridSearchCV,cross_val_score
from sklearn.preprocessing import MinMaxScaler, label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.naive_bayes import GaussianNB

In [105]:
car=pd.read_csv('C:/Users/Roman Lyskov/ML/car_evaluation.csv',na_values=['?'])

In [106]:
y = car.iloc[:,6]
X = car.iloc[:,:6]

In [107]:
# Check NA
X[X.isna().any(axis = 1)]

,vhigh,vhigh.1,2,2.1,small,low


In [108]:
y.value_counts()

unacc    1209
acc       384
good       69
vgood      65
Name: unacc, dtype: int64

In [109]:
car.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [110]:
car = car.rename(columns={'vhigh': '0'})

In [111]:
car = car.rename(columns={'vhigh.1': '1'})

In [112]:
car = car.rename(columns={'2.1': '3'})

In [113]:
car = car.rename(columns={'small': '4'})

In [114]:
car = car.rename(columns={'low': '5'})

In [115]:
car = car.rename(columns={'unacc': '6'})

In [116]:
car.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [117]:
X.iloc[:,0:2].replace({'low':0,'med':1/3,'high':2/3, 'vhigh':1}, inplace = True)
X.iloc[:,2].replace({'2':0,'3':1/3,'4':2/3,'5more':1},inplace = True)
X.iloc[:,3].replace({'2':0,'4':0.5,'more':1},inplace = True)
X.iloc[:,4].replace({'small':0,'med':0.5,'big':1},inplace = True)
X.iloc[:,5].replace({'low':0,'med':0.5,'high':1},inplace = True)

C:\Users\Roman Lyskov\anaconda3\lib\site-packages\pandas\core\frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=45)
f_measure_score = {'decision_tree':{},'knn':{}, 'rand_forest':{}}

In [119]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=45)

# Decision tree

In [120]:
tree_model = DecisionTreeClassifier()

In [121]:
para_tree = {'criterion':['entropy'],'max_depth':list(range(8,11)),'min_samples_leaf':list(range(1,4)),'random_state':[45]}

In [122]:
grid_tree = GridSearchCV(tree_model, para_tree, cv = cv, scoring='f1_micro')
grid_tree.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=45, shuffle=True),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
       

In [123]:
y_pred_tree = grid_tree.predict(X_test)
nested_score_tree = cross_val_score(grid_tree, X = X, y = y, cv = cv) 
f_measure_score['decision_tree']['mean'] = np.mean(nested_score_tree)
f_measure_score['decision_tree']['std'] = np.std(nested_score_tree)

In [124]:
print('precision,recall,f-measure\n', classification_report(y_test,y_pred_tree))

precision,recall,f-measure
               precision    recall  f1-score   support

         acc       0.93      0.95      0.94        83
        good       0.72      0.87      0.79        15
       unacc       0.99      0.98      0.99       230
       vgood       1.00      0.83      0.91        18

    accuracy                           0.96       346
   macro avg       0.91      0.91      0.91       346
weighted avg       0.97      0.96      0.96       346



In [125]:
grid_tree.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'random_state': 45}

# K NEIGHBOR

In [126]:
para_knn = {'n_neighbors':list(range(12,17))}
knn = KNeighborsClassifier()
grid_knn = GridSearchCV(knn, para_knn, cv = cv, scoring='f1_micro')
grid_knn.fit(X_train,y_train)
y_pred_knn = grid_knn.predict(X_test)
nested_score_knn = cross_val_score(grid_knn, X = X, y = y, cv = cv) 
f_measure_score['knn']['mean'] = np.mean(nested_score_knn)
f_measure_score['knn']['std'] = np.std(nested_score_knn)

In [127]:
print('precision,recall,f-measure\n', classification_report(y_test,y_pred_knn))

precision,recall,f-measure
               precision    recall  f1-score   support

         acc       0.81      0.98      0.89        83
        good       0.78      0.47      0.58        15
       unacc       0.99      0.97      0.98       230
       vgood       1.00      0.72      0.84        18

    accuracy                           0.93       346
   macro avg       0.89      0.78      0.82       346
weighted avg       0.94      0.93      0.93       346



In [128]:
grid_knn.best_params_

{'n_neighbors': 15}

# Random Forest

In [129]:
from sklearn.ensemble import RandomForestClassifier

In [134]:
forest_model = RandomForestClassifier()
params = {'max_depth': np.arange(1, 5), 'min_samples_leaf' : np.arange(1, 5)}
grid_forest = GridSearchCV(forest_model, params, cv = cv, scoring='f1_micro')
grid_forest.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=45, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                           

In [135]:
y_pred_forest = grid_forest.predict(X_test)
nested_score_forest = cross_val_score(grid_forest, X = X, y = y, cv = cv) 
f_measure_score['rand_forest']['mean'] = np.mean(nested_score_forest)
f_measure_score['rand_forest']['std'] = np.std(nested_score_forest)

In [136]:
print('precision,recall,f-measure\n', classification_report(y_test,y_pred_forest))

precision,recall,f-measure
               precision    recall  f1-score   support

         acc       0.64      0.84      0.73        83
        good       0.00      0.00      0.00        15
       unacc       0.95      0.97      0.96       230
       vgood       0.00      0.00      0.00        18

    accuracy                           0.85       346
   macro avg       0.40      0.45      0.42       346
weighted avg       0.78      0.85      0.81       346



C:\Users\Roman Lyskov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [137]:
grid_forest.best_params_

{'max_depth': 4, 'min_samples_leaf': 3}

# Model Comparision

In [138]:
for k,v in f_measure_score.items():
    print(k, ': ', v)

decision_tree :  {'mean': 0.9716124479096653, 'std': 0.010229839245264879}
knn :  {'mean': 0.949633687323565, 'std': 0.0121022836033064}
rand_forest :  {'mean': 0.8720022852533942, 'std': 0.016728419170906072}


Дерево решений — наилучший метод из испробованных в рамках решения данной задачи.